Notes
- Need to re-check the Horn's test implementation
- Not sure if later steps should keep the column scaling from the Horn's test step

In [ ]:
using CSV
using DataFrames
using LinearAlgebra
using Statistics
using StatsBase
using Distributions
using Plots; gr()
using LaTeXStrings
using Printf

In [ ]:
da1 = open("derived-dataframes/regression-data-v5/conv-annotator_code-speaker_counts.csv") do io
    CSV.read(io, DataFrame)
end

In [ ]:
gda1 = groupby(da1, [:document, :conversation_number])
da2 = combine(gda1, names(da1)[5:end] .=> median; renamecols=false)
outcomes = combine(gda1, :outcome .=> mode; renamecols=false)[:, "outcome"]
insertcols!(da2, 3, :outcome => outcomes)

In [ ]:
X = Matrix(da2[:, 4:end])

In [ ]:
# square root the matrix
Xt = sqrt.(X)

# center the matrix
Xt .-= mean(Xt)

colmeans = mean(Xt ; dims=1)
rowmeans = mean(Xt ; dims=2)

Xt .-= colmeans
Xt .-= rowmeans

In [ ]:
maximum(mean(Xt ; dims=1)), minimum(mean(Xt ; dims=1))

In [ ]:
maximum(mean(Xt ; dims=2)), minimum(mean(Xt ; dims=2))

In [ ]:
# take the decomposition
U, s, V = svd(Xt)

In [ ]:
# eyeball how the dominant singular values behave
s[1:10]

In [ ]:
# we want to check that these are uncorrelated (output should be close to zero)
print(cor(U[:, 1], U[:, 2]), "\n", cor(V[:, 1], V[:, 2]))

In [ ]:
# we also want to check that the matrices are centered (output should be close to zero)
print(extrema(mean(U ; dims=1)[1:10]), "\n", extrema(mean(V ; dims=1)[1:10]))

In [ ]:
sum(s.<=1e-12) # check how many are basically zero

In [ ]:
length(s)

### Horn's test for number of factors
https://link.springer.com/article/10.1007/BF02289447

Source: https://blogs.sas.com/content/iml/2020/12/09/parallel-analysis-retain-principal-components.html

In [ ]:
# standardize the matrix columns (multiply through by 1/sqrt(sample column variance))
colvars = mean(Xt .^ 2 ; dims=1)
Z = Xt ./ sqrt.(colvars)

In [ ]:
var(Z; dims=1)

In [ ]:
# take the correlation matrix
R = cor(Z)
N, p = size(Z)

# get the eigenvalues (note it's a scaled gram matrix so PSD, so all are nonnegative reals)
evals = eigvals(R)[end:-1:1] # sort in the usual way (highest to lowest)

In [ ]:
# Wishart(ν, S) where ν = DoF and S = pxp scale matrix
dist = Wishart(N - 1, Matrix(I, p, p))

nsim = 5000
simS = rand(dist, nsim) ./ (N - 1)

In [ ]:
# https://documentation.sas.com/doc/en/pgmsascdc/9.4_3.3/imlug/imlug_langref_sect091.html
simsdinv = inv.(sqrt.(Diagonal.(diag.(simS))))
simR = simsdinv .* simS .* simsdinv

In [ ]:
# keep going
simevals = eigvals.(simR)
simevals = stack(simevals, dims=2)[end:-1:1, :] # reshape and reverse sort

In [ ]:
# p-value and test
alpha = 0.05
simevals = [quantile(simevals[i, :], 1-alpha, sorted=false) for i in 1:p]

In [ ]:
plot(; title="Scree", xaxis="Index", yaxis="Eigenvalue")
plot!(1:p, evals; color=1, linewidth=2, label="Observed")
#scatter!(1:p, evals; color=1, alpha=0.5, markerstrokewidth=0, label="")
plot!(1:p, simevals; color=2, linewidth=2, label="Simulated")
#scatter!(1:p, simevals; color=2, alpha=0.5, markerstrokewidth=0, label="")

In [ ]:
nfactors = 0
for k in 1:p
    if simevals[k] > evals[k]
        nfactors = k - 1
        print(nfactors)
        break
    end
end

In [ ]:
# output is 5, so we'll keep 5 factors

### Make scaled version of the SVD

In [ ]:
Ut = U * Diagonal(sqrt.(s))
Vt = V * Diagonal(sqrt.(s))

## Visualizations

### Look at the singular values

In [ ]:
# scree
scatter(1:50, s[1:50]; color=7, label=L"\sigma")
plot!(; title="First 50 singular values", xaxis=L"j", yaxis=L"s_j")

In [ ]:
# semi log the above
scatter(1:50, log.(s[1:50]); color=7, label=L"\sigma")
plot!(; title="First 50 singular values on semi-log scale", xaxis=L"j", yaxis=L"\log (s_j)")

In [ ]:
# do singular values follow a power law relationship?
scatter((log.(1:50)), (log.(s[1:50])); color=7, label=L"\sigma")
plot!(; title="First 50 singular values on log scale", xaxis=L"\log (j)", yaxis=L"\log (s_j)")

### Look at the dominant two singular vectors

In [ ]:
# biplot
plot(; title="Biplot: Singular vectors 1 & 2", xaxis="First singular vector", yaxis="Second singular vector")
scatter!(Ut[:, 1], Ut[:, 2]; color=:violet, label="Conversation")
scatter!(Vt[:, 1], Vt[:, 2]; color=3, label="Code-speaker")

In [ ]:
# code-speaker, color-coded by speaker
plot(; title="Code-speakers: Singular vectors 1 & 2", xaxis="First singular vector", yaxis="Second singular vector")
scatter!(Vt[1:2:end, 1], Vt[1:2:end, 2]; color=:yellow, label="Helper")
scatter!(Vt[2:2:end, 1], Vt[2:2:end, 2]; color=:blue, label="Learner")

In [ ]:
# code-speaker, color-coded by code type (highest-level category)
codegroups = [x[1] for x in split.([x[3:end] for x in names(da1)[5:end]], " > ")]
plt = plot(; title="Code-speakers: Singular vectors 1 & 2", xaxis="First singular vector", yaxis="Second singular vector")
    
for grp in unique(codegroups)
    flag = codegroups .== grp
    scatter!(Vt[flag, 1], Vt[flag, 2]; label=grp)
end
display(plt)

In [ ]:
# conversation, color-coded by outcome
successes = da2[:, 3] .== "S"
plot(; title="Conversations: Singular vectors 1 & 2", xaxis="First singular vector", yaxis="Second singular vector")
scatter!(Ut[successes, 1], Ut[successes, 2]; color=1, label="Success")
scatter!(Ut[.!successes, 1], Ut[.!successes, 2]; color=2, label="Failure")

In [ ]:
# Test* whether the dominant left singular vector represents conversation length
plot(; title="Conversations: First singular vector vs no. observations", titlefontsize=11)
plot!(; xaxis="Number of observations", yaxis="Variable score")
scatter!(sum(X; dims=2), Ut[:, 1]; color=:violet, label="Conversation")

In [ ]:
# Test* whether the dominant right singular vector represents code-speaker frequency
plot(; title="Code-speakers: First singular vector vs no. observations", titlefontsize=11)
plot!(; xaxis="Number of observations", yaxis="Variable score")
scatter!(sum(X; dims=1)[1, :], Vt[:, 1]; color=3, label="Code-speaker")

In [ ]:
# Test* whether the 2nd left singular vector represents conversation length
plot(; title="Conversations: Second singular vector vs no. observations", titlefontsize=11)
plot!(; xaxis="Number of observations", yaxis="Variable score")
scatter!(sum(X; dims=2), Ut[:, 2]; color=:violet, label="Conversation")

In [ ]:
# Test* whether the 2nd right singular vector represents code-speaker frequency
plot(; title="Code-speakers: Second singular vector vs no. observations", titlefontsize=11)
plot!(; xaxis="Cube-root-transformed number of observations", yaxis="Variable score")
scatter!(sum(X; dims=1)[1, :].^0.33, Vt[:, 2]; color=3, label="Code-speaker")

### Analyze the correlations for the dominant singular vectors

In [ ]:
th = 2/sqrt(length(Ut[:, 1])) # this is the threshold for significance of conversation-annotator sized vectors

# BH correction
th = quantile(Normal(0.0, 1.0), 1 - (0.025 / nfactors))/sqrt(length(Ut[:, 1]))

In [ ]:
cor(successes, rowmeans[:, 1]) # the additive conversation main effect *is not* significantly correlated with outcomes

In [ ]:
for i in 1:nfactors
    c = cor(successes, Ut[:, i])
    if c > th
        print("**Singular vector no. ", i, " is significantly (positively) correlated with outcomes (", c, ")\n")
    elseif c < -th
        print("**Singular vector no. ", i, " is significantly (negatively) correlated with outcomes (", c, ")\n")
    else
        print("Singular vector no. ", i, " is not significantly correlated with outcomes (", c, ")\n")
    end
end

In [ ]:
convlen = sum(X; dims=2)[:, 1]

In [ ]:
cor(convlen, rowmeans[:, 1]) # the additive conversation main effect *is* significantly correlated with length

In [ ]:
for i in 1:nfactors
    c = cor(convlen, Ut[:, i])
    if c > th
        print("**Singular vector no. ", i, " is significantly (positively) correlated with conv length (", c, ")\n")
    elseif c < -th
        print("**Singular vector no. ", i, " is significantly (negatively) correlated with conv length (", c, ")\n")
    else
        print("Singular vector no. ", i, " is not significantly correlated with conv length (", c, ")\n")
    end
end

In [ ]:
codefreq = sum(X; dims=1)[1, :]
cor(codefreq, colmeans[1, :]) # the additive code-speaker main effect *is* significantly correlated with code frequency

### Look at the dominant second and third singular vectors

In [ ]:
# 2-3 biplot (dropping the first singular vector because it is highly 
# correlated with length but not outcome, so we feel like we can explain it)
plot(; title="Biplot: Singular vectors 2 & 3", xaxis="Second singular vector", yaxis="Third singular vector")
scatter!(Ut[successes, 2], Ut[successes, 3]; color=1, label="Conversation (S)")
scatter!(Ut[.!successes, 2], Ut[.!successes, 3]; color=2, label="Conversation (F)")
scatter!(Vt[:, 2], Vt[:, 3]; color=3, label="Code-speaker")

In [ ]:
# Now just look at the conversations, for clarity. Based on the 
# correlations, successes should go in the bottom left corner (quadrant
# III) and failures should go in the top right corner (quadrant I).
# You can also kind of see that the red dot cloud is slightly below and 
# to the right of the blue dot cloud
plot(; title="Conversations: Singular vectors 2 & 3", xaxis="Second singular vector", yaxis="Third singular vector")
scatter!(Ut[successes, 2], Ut[successes, 3]; color=1, label="Success")
scatter!(Ut[.!successes, 2], Ut[.!successes, 3]; color=2, label="Failure")

In [ ]:
# Code-speakers follow the analogous relationship, we we want to identify 
# the corresponding points from the 2-3 biplot
plotlyjs()

plt = plot()
scatter!(Ut[successes, 2], Ut[successes, 3]; color=1, hover="", label="Conversation (S)", ma=0.7)
scatter!(Ut[.!successes, 2], Ut[.!successes, 3]; color=2, hover="", label="Conversation (F)", ma=0.7)
scatter!(Vt[:, 2], Vt[:, 3]; color=3, hover=names(da1)[5:end], label="Code-speaker", ma=0.7)

xlabel!("Second singular vector")
ylabel!("Third singular vector")
title!("Counts SVD biplot")

display(plt)
gr()

### Look at the dominant second and fourth singular vectors

In [ ]:
# 2-4 biplot
# Based on the correlations, successes should go in the top left corner
# (quadrant II) and failures should go in the bottom right corner (quadrant IV)
plot(; title="Biplot: Singular vectors 2 & 4", xaxis="Second singular vector", yaxis="Fourth singular vector")
scatter!(Ut[successes, 2], Ut[successes, 4]; color=1, label="Conversation (S)")
scatter!(Ut[.!successes, 2], Ut[.!successes, 4]; color=2, label="Conversation (F)")
scatter!(Vt[:, 2], Vt[:, 4]; color=3, label="Code-speaker")

In [ ]:
# Look at the plot if you don't believe me
plot(; title="Conversations: Singular vectors 2 & 4", xaxis="Second singular vector", yaxis="Fourth singular vector")
scatter!(Ut[successes, 2], Ut[successes, 4]; color=1, label="Success")
scatter!(Ut[.!successes, 2], Ut[.!successes, 4]; color=2, label="Failure")

In [ ]:
# Identify corresponding codes
plotlyjs()

plt = plot()
scatter!(Ut[successes, 2], Ut[successes, 4]; color=1, hover="", label="Conversation (S)", ma=0.7)
scatter!(Ut[.!successes, 2], Ut[.!successes, 4]; color=2, hover="", label="Conversation (F)", ma=0.7)
scatter!(Vt[:, 2], Vt[:, 4]; color=3, hover=names(da1)[5:end], label="Code-speaker", ma=0.7)

xlabel!("Second singular vector")
ylabel!("Fourth singular vector")
title!("Counts SVD biplot")

display(plt)
gr()

### Look at the dominant third and fourth singular vectors

In [ ]:
# 3-4 biplot
# Based on the correlations, successes should go in the top left corner
# (quadrant II) and failures should go in the bottom right corner (quadrant IV)
plot(; title="Biplot: Singular vectors 3 & 4", xaxis="Third singular vector", yaxis="Fourth singular vector")
scatter!(Ut[successes, 3], Ut[successes, 4]; color=1, label="Conversation (S)")
scatter!(Ut[.!successes, 3], Ut[.!successes, 4]; color=2, label="Conversation (F)")
scatter!(Vt[:, 3], Vt[:, 4]; color=3, label="Code-speaker")

In [ ]:
# Look at the plot if you don't believe me
plot(; title="Conversations: Singular vectors 3 & 4", xaxis="Third singular vector", yaxis="Fourth singular vector")
scatter!(Ut[successes, 3], Ut[successes, 4]; color=1, label="Success")
scatter!(Ut[.!successes, 3], Ut[.!successes, 4]; color=2, label="Failure")

In [ ]:
# Identify corresponding codes
plotlyjs()

plt = plot()
scatter!(Ut[successes, 3], Ut[successes, 4]; color=1, hover="", label="Conversation (S)", ma=0.7)
scatter!(Ut[.!successes, 3], Ut[.!successes, 4]; color=2, hover="", label="Conversation (F)", ma=0.7)
scatter!(Vt[:, 3], Vt[:, 4]; color=3, hover=names(da1)[5:end], label="Code-speaker", ma=0.7)

xlabel!("Third singular vector")
ylabel!("Fourth singular vector")
title!("Counts SVD biplot")

display(plt)
gr()

#### Orthogonal rotation so the horizontal axis represents the outcome gradient

In [ ]:
# regression something something

### Alternative approach: Induce sparsity with Varimax algorithm

In [ ]:
function varimax(X::T; maxiter::Int=1000, eps::Float64=1e-6) where {T<:AbstractMatrix}

    G = copy(X)
    m, p = size(G)
    Q = Matrix{Float64}(I(p))
    if p < 2
        return G, Q
    end

    d = 0.0
    for i in 1:maxiter
        z = G * Q
        cs = sum(abs2, z; dims=1)[:]
        dcs = Diagonal(cs / m)
        B = G' * (z.^3 - z * dcs)
        ss = svd(B)
        Q .= ss.U * ss.V'
        dlast = d
        d = sum(ss.S)
        if d < dlast * (1 + eps)
            break
        end
    end

    G .= G * Q

    # Rescale so that the largest loading is positive
    for j in 1:size(G, 2)
        _, ii = findmax(abs2, G[:, j])
        if G[ii, j] < 0
            G[:, j] = -G[:, j]
            Q[:, j] = -Q[:, j]
        end
    end

    return G, Q
end

In [ ]:
jj = 2:nfactors
m = length(jj)

In [ ]:
Urot, QUrot = varimax(U[:, jj])
Urot .*= sqrt(N)
Vrot, QVrot = varimax(V[:, jj])
Vrot .*= sqrt(p)
Srot = QUrot' * Diagonal(s[jj]) * QVrot ./ sqrt(N*p)

#### Check what happened to the sparsity (kurtosis)

In [ ]:
for i in 1:m
    print(@sprintf "U%d Kurtosis: %f.4 \t %f.4\n" i kurtosis(U[:, i]) kurtosis(Urot[:, i]))
    print(@sprintf "V%d Kurtosis: %f.4 \t %f.4\n\n" i kurtosis(V[:, i]) kurtosis(Vrot[:, i]))
end

#### Check what happened to the correlations

In [ ]:
for i in 1:m
    c = cor(successes, Urot[:, i])
    if c > th
        print("**Score vector no. ", i, " is significantly (positively) correlated with outcomes (", c, ")\n")
    elseif c < -th
        print("**Score vector no. ", i, " is significantly (negatively) correlated with outcomes (", c, ")\n")
    else
        print("Score vector no. ", i, " is not significantly correlated with outcomes (", c, ")\n")
    end
end

In [ ]:
for i in 1:m
    c = cor(convlen, Urot[:, i])
    if c > th
        print("**Score vector no. ", i, " is significantly (positively) correlated with conv length (", c, ")\n")
    elseif c < -th
        print("**Score vector no. ", i, " is significantly (negatively) correlated with conv length (", c, ")\n")
    else
        print("Score vector no. ", i, " is not significantly correlated with conv length (", c, ")\n")
    end
end

In [ ]:
j = 2 # index of the significant score vector

#### Check for improved interpretability in the significant loading vector(s)

In [ ]:
histogram(Vrot[:, j]; label="", title=(@sprintf "Sparsified factor %d loadings" j), xaxis="Loading", yaxis="Frequency")

In [ ]:
# get outlier indices
k = 2
iipos = [i for i in 1:p][Vrot[:, j] .> k]
iineg = [i for i in 1:p][Vrot[:, j] .< -k]

In [ ]:
# get corresponding variables
# positive loadings on negative correlation, so these are bad for outcome
names(da1)[5:end][iipos]

In [ ]:
# these are good for outcome (wtf?)
names(da1)[5:end][iineg]

## Raw counts of things

In [ ]:
codecountls = sum(X[:, 1:2:end] .+ X[:, 2:2:end] ; dims=1)[1, :]

In [ ]:
annls = [v[1][3:end] for v in split.(names(da1)[5:2:end], "\', \'")]

for ann in annls
    print(ann, "\n")
end

In [ ]:
for k in 11:14
    print(annls[k], repeat(" ", 75-length(annls[k])), codecountls[k], "\n")
end

In [ ]:
for k in 16:23
    print(annls[k], repeat(" ", 75-length(annls[k])), codecountls[k], "\n")
end

In [ ]:
for k in 24:26
    print(annls[k], repeat(" ", 75-length(annls[k])), codecountls[k], "\n")
end

In [ ]:
for k in 27:41
    print(annls[k], repeat(" ", 75-length(annls[k])), codecountls[k], "\n")
end

In [ ]:
for k in 42:46
    print(annls[k], repeat(" ", 75-length(annls[k])), codecountls[k], "\n")
end

In [ ]:
for k in 47:48
    print(annls[k], repeat(" ", 75-length(annls[k])), codecountls[k], "\n")
end

## Sandbox

In [ ]:
# sphericity isn't applicable -- here's why
# covariance matrix and data dimensions
R = cor(Xt)
N, p = size(Xt)

# Bartlett's sphericity statistic
# T=−log(det(R))(N−1−(2p+5)/6)
det(R) # is zero, so Bartlett's not valid

In [ ]:
plotlyjs()
#, extra_plot_kwargs = KW(:include_mathjax => "cdn")
x = range(0, 10, length=100)
y = sin.(x)
y_noisy = @. sin(x) + 0.1*randn()

# this plots into a standalone window via Plotly
tmp = plot(x, y, label="sin(x)", lc=:black, lw=2)
scatter!(x, y_noisy, label="data", mc=:red, ms=2, ma=0.5)
plot!(legend=:bottomleft)
title!("Sine with noise, plotted with Plotly")
xlabel!("x")
ylabel!("y")
display(tmp)
gr()

In [ ]:
# plotly latex stupidity (need to figure out how to set defaults)
# https://docs.juliaplots.org/latest/backends/#MathJax

In [ ]:
# https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/

#using GLMakie
# I never figured out the c++ thing -- maybe revisit this once things are updated?
# https://github.com/MakieOrg/Makie.jl/tree/master/GLMakie#troubleshooting-opengl.
# https://github.com/microsoft/WSL/issues/2855#issuecomment-358861903
# https://discourse.julialang.org/t/failed-to-precompile-makie/20758
# https://discourse.julialang.org/t/how-to-use-glmakie-in-jupyter-notebook/83552
# https://docs.makie.org/v0.21/tutorials/basic-tutorial#Getting-started-with-Makie
# https://docs.makie.org/v0.21/

# ]build GLMakie

# import Pkg; Pkg.add("PlotlyJS")